In [1]:
import sys
sys.path.append('../../30_data_tools/')

In [2]:
import numpy as np
import cv2
from PIL import Image, ImageFilter
import plotly.express as px
from tqdm import tqdm

In [3]:
import subprocess
import json

In [4]:
from perlin_noise import PerlinNoise

In [5]:
from helper import load_dotenv

In [6]:
dotenv = load_dotenv()

In [7]:
def get_perlin_noise( img_size, octaves, seed, deformation_factor=(1,1) ):
    noise = PerlinNoise(octaves=octaves, seed=seed)
    pic = [[noise([i/img_size[0] * deformation_factor[0], j/img_size[1] * deformation_factor[1]]) for j in range(img_size[0])] for i in tqdm(range(img_size[1]))]
    random_noise = (np.array(pic) * 0.5 + 0.5)
    random_noise -= random_noise.min()
    random_noise /= random_noise.max()

    return random_noise

def adjust_contrast( img, contrast_range ):
    img *= contrast_range
    img += (1 - contrast_range) * 0.5

    return img

In [8]:
def render_pattern_images( input_path, output_path, dot_style ):
    res = subprocess.run(
        f'osascript ./convert_to_screen_image.sctp "{ input_path }" "{ output_path }" "{ dot_style }"',
        shell=True,
        capture_output=True
    )

# Perlin Noise images errechnen

In [9]:
octaves = 5
seed = 33

original_noise = get_perlin_noise((2500,2500), octaves, seed, deformation_factor=(1,0.1))

for i in range(10):
    contrast_range = 1 - (i / 10)
    adjusted_img = adjust_contrast(
        original_noise.copy(),
        contrast_range
    )

    img = Image.fromarray((adjusted_img * 255).round().astype('uint8'))
    out_path = dotenv['PATTERN_DIR'] / f'perlin_noise.{ octaves }.{ seed }.{ str(contrast_range)[2:4] }.df1;01.jpg'
    img = img.resize((6000,6000))
    img.save( out_path, progressive=True, dpi=(600,600) )

100%|███████████████████████████████████████| 2500/2500 [01:08<00:00, 36.29it/s]


# Rastern

In [10]:
perlin_images = list(dotenv['PATTERN_DIR'].glob('./perlin_noise*.jpg'))

In [11]:
for perlin_img_path in tqdm(perlin_images):
    ellipse_img_path = dotenv['PATTERN_DIR'] / f'{ perlin_img_path.name.replace(perlin_img_path.suffix,"") }.150pli.ellipse'
    linie_img_path = dotenv['PATTERN_DIR'] / f'{ perlin_img_path.name.replace(perlin_img_path.suffix,"") }.150pli.linie'
    
    render_pattern_images(
        perlin_img_path,
        ellipse_img_path,
        'ellipse.150lpi'
    )
    
    render_pattern_images(
        perlin_img_path,
        linie_img_path,
        'linie.150lpi'
    )

100%|███████████████████████████████████████████| 20/20 [00:53<00:00,  2.69s/it]


# generate control json

In [ ]:
with (dotenv['PATTERN_DIR'] / 'available_patterns.json').open('w') as json_file:
    json_file.write(json.dumps([
        entry.name 
        for entry in dotenv['PATTERN_DIR'].glob('./*.tif')
        if entry.name.startswith('diamons') == False
    ]))